In [3]:

from balm.data import load_dataset, DataCollator
from balm.models.balm import BalmForMaskedLM
from balm.models.balm_moe import BalmMoEForMaskedLM
from balm.models.balm_moe_rope import BalmMoERoPEForMaskedLM
from balm.tokenizer import Tokenizer
from balm.training.trainer import Trainer


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [4]:
tokenizer = Tokenizer(vocab="./vocab.json")

In [5]:
def remove_sep(txt):
    return txt.replace("</s>", "<cls><cls>")


data_files = {
    "train": "./balm/test_data/test.txt",
    "test": "./balm/test_data/test_1k.txt",
    "eval": "./balm/test_data/test_1k.txt",
}

dataset = load_dataset("text", data_files=data_files, preprocess_fn=remove_sep)

In [6]:
tokenized_dataset = dataset.map(
    lambda x: tokenizer(
        x["text"],
        padding=True,
        truncation=True,
        max_length=320,
    ),
    remove_columns="text"
)

  0%|          | 0/66792 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
collator = DataCollator(tokenizer=tokenizer)

In [8]:
# model = BalmMoERoPEForMaskedLM(
model = BalmMoEForMaskedLM(
    embed_dim=256,
    ffn_dim=1024,
    num_experts=4,
    num_layers=8,
    num_heads=8,
    expert_capacity=128,
    router_z_loss_coef=0.01,
    router_aux_loss_coef=0.01,
    vocab_size=tokenizer.vocab_size,
)


In [9]:
model.num_parameters

18982689

## Trainer todos:
- [x] train/eval dataloader
- [x] move everything to the correct device (cuda or cpu)
- [x] learning rate (including scheduler)
- [ ] gradient accumulation steps
- [ ] logging
- [ ] checkpointing
- [ ] distributed training
- [ ] gradient clipping



In [10]:
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    matthews_corrcoef,
    precision_recall_fscore_support,
)


def compute_metrics(eval_predictions):
    # # Convert predictions and labels to CPU if necessary and to NumPy arrays for sklearn
    # preds = eval_predictions.predictions.cpu().numpy()
    # lbls = eval_predictions.labels.cpu().numpy()

    # # Calculate accuracy
    # accuracy = accuracy_score(lbls, preds)

    # # Calculate AUROC
    # # Note: For multi-class, you need to binarize the labels and use `average='macro'` or `average='micro'`
    # try:
    #     auroc_score = roc_auc_score(
    #         lbls, preds, multi_class="ovo" if len(lbls.shape) > 1 else "raise"
    #     )
    # except ValueError:
    #     # AUROC is not defined when there is only one class in the labels, or labels are not binary
    #     auroc_score = float("nan")

    # # Calculate Matthews correlation coefficient (MCC)
    # try:
    #     mcc_score = matthews_corrcoef(lbls, preds)
    # except ValueError:
    #     # MCC is not defined when there is only one class in the labels
    #     mcc_score = float("nan")

    # # Calculate precision, recall, and F1-score
    # precision, recall, f1, _ = precision_recall_fscore_support(
    #     lbls, preds, average="binary"
    # )

    # Calculate perplexity if logits are provided and labels are not one-hot encoded
    if hasattr(eval_predictions, "logits"):
        softmaxes = F.softmax(eval_predictions.logits, dim=-1)
        cross_entropy = F.cross_entropy(softmaxes, eval_predictions.labels)
        perplexity = torch.exp(cross_entropy)

    return {
        "perplexity": perplexity.item(),
        # "accuracy": accuracy,
        # "auroc": auroc_score,
        # "mcc": mcc_score,
        # "precision": precision,
        # "recall": recall,
        # "f1": f1,
    }

In [11]:
trainer = Trainer(
    model=model, 
    data_collator=collator, 
    train_dataset=tokenized_dataset["train"], 
    eval_dataset=tokenized_dataset["eval"],
    epochs=1,
    logging_steps=10,
    eval_steps=50,
    warmup_steps = 50,
    per_device_train_batch_size=32,
    # per_device_eval_batch_size=32,
    use_cpu=True,
    # compute_metrics=compute_metrics,
)



In [14]:
trainer.device

device(type='cpu')

In [15]:
trainer.train()

  0%|          | 0/2087 [00:00<?, ?it/s]

step 10   | loss: 3.0613 | lm_loss: 3.0290 | router_z_loss: 0.0220 | router_aux_loss: 0.0102 | lr: 0.000080
step 20   | loss: 2.7867 | lm_loss: 2.7573 | router_z_loss: 0.0194 | router_aux_loss: 0.0101 | lr: 0.000160
step 30   | loss: 2.6294 | lm_loss: 2.6043 | router_z_loss: 0.0150 | router_aux_loss: 0.0101 | lr: 0.000240
step 40   | loss: 2.5059 | lm_loss: 2.4840 | router_z_loss: 0.0117 | router_aux_loss: 0.0101 | lr: 0.000320
step 50   | loss: 2.3115 | lm_loss: 2.2930 | router_z_loss: 0.0084 | router_aux_loss: 0.0101 | lr: 0.000400


0it [00:00, ?it/s]

<<< EVAL >>> loss: 2.2903
step 60   | loss: 2.2155 | lm_loss: 2.1983 | router_z_loss: 0.0070 | router_aux_loss: 0.0101 | lr: 0.000398
step 70   | loss: 2.0435 | lm_loss: 2.0280 | router_z_loss: 0.0055 | router_aux_loss: 0.0100 | lr: 0.000396
step 80   | loss: 2.1350 | lm_loss: 2.1198 | router_z_loss: 0.0053 | router_aux_loss: 0.0099 | lr: 0.000394
step 90   | loss: 2.0281 | lm_loss: 2.0132 | router_z_loss: 0.0049 | router_aux_loss: 0.0099 | lr: 0.000392
step 100  | loss: 2.0571 | lm_loss: 2.0427 | router_z_loss: 0.0045 | router_aux_loss: 0.0099 | lr: 0.000390


0it [00:00, ?it/s]

...

<<< EVAL >>> loss: 0.3863
step 1960 | loss: 0.3871 | lm_loss: 0.3753 | router_z_loss: 0.0019 | router_aux_loss: 0.0099 | lr: 0.000025
step 1970 | loss: 0.3896 | lm_loss: 0.3779 | router_z_loss: 0.0018 | router_aux_loss: 0.0099 | lr: 0.000023
step 1980 | loss: 0.4136 | lm_loss: 0.4020 | router_z_loss: 0.0017 | router_aux_loss: 0.0099 | lr: 0.000021
step 1990 | loss: 0.3407 | lm_loss: 0.3290 | router_z_loss: 0.0018 | router_aux_loss: 0.0099 | lr: 0.000019
step 2000 | loss: 0.4843 | lm_loss: 0.4727 | router_z_loss: 0.0018 | router_aux_loss: 0.0099 | lr: 0.000017


0it [00:00, ?it/s]

<<< EVAL >>> loss: 0.3835
step 2010 | loss: 0.4303 | lm_loss: 0.4185 | router_z_loss: 0.0018 | router_aux_loss: 0.0099 | lr: 0.000015
step 2020 | loss: 0.3774 | lm_loss: 0.3659 | router_z_loss: 0.0016 | router_aux_loss: 0.0099 | lr: 0.000013
step 2030 | loss: 0.3979 | lm_loss: 0.3862 | router_z_loss: 0.0018 | router_aux_loss: 0.0099 | lr: 0.000011
step 2040 | loss: 0.3444 | lm_loss: 0.3329 | router_z_loss: 0.0017 | router_aux_loss: 0.0098 | lr: 0.000009
step 2050 | loss: 0.3665 | lm_loss: 0.3548 | router_z_loss: 0.0018 | router_aux_loss: 0.0099 | lr: 0.000007


0it [00:00, ?it/s]

<<< EVAL >>> loss: 0.3885
step 2060 | loss: 0.4335 | lm_loss: 0.4221 | router_z_loss: 0.0016 | router_aux_loss: 0.0098 | lr: 0.000005
step 2070 | loss: 0.3745 | lm_loss: 0.3629 | router_z_loss: 0.0017 | router_aux_loss: 0.0099 | lr: 0.000003
step 2080 | loss: 0.3213 | lm_loss: 0.3097 | router_z_loss: 0.0017 | router_aux_loss: 0.0099 | lr: 0.000001
Training complete


In [13]:
len(trainer.eval_dataset) // trainer.total_eval_batch_size



31

In [10]:
train_dataloader = DataLoader(
    tokenized_dataset["train"],
    batch_size=trainer.total_train_batch_size,
    shuffle=True,
)




In [11]:
for batch in train_dataloader:
    collated = collator(batch)
    break


In [12]:
collated

{'input_ids': tensor([[ 0,  9,  7,  ...,  1,  1,  1],
         [ 0, 32,  7,  ...,  1,  1,  1],
         [ 0,  9,  7,  ...,  1,  1,  1],
         ...,
         [ 0, 16,  7,  ...,  1,  1,  1],
         [ 0,  9,  7,  ...,  1,  1,  1],
         [ 0, 32,  7,  ...,  1,  1,  1]]),
 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100],
         [-100,   16, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         ...,
         [-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         [-100,   16, -100,  ..., -100, -100, -100]]),
 'key_padding_mask': tensor([[False, False, False,  ...,  True,  True,  True],
         [False, False, False,  ...,  True,  True,  True],
         [False, False, False,  ...,  True,  True,  True],
         ...,
         [False, False, False,  ...,  True,  True,  True],
         [False, False, False,  ...,  True,  True,  True],
         [False, False, False,  ...,  True,  True,  